# Project | Natural Language Processing
### News Headline: Real or Fake News

### Prepare the environment

In [3]:
%pip install pandas numpy scikit-learn nltk spacy xgboost matplotlib seaborn transformers datasets sentence-transformers streamlit
from sympy import python
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
# python -m nltk.downloader stopwords punkt wordnet

# python -m spacy download en_core_web_sm
#%python -m spacy download en_core_web_sm
import spacy.cli
spacy.cli.download("en_core_web_sm")
import spacy
nlp = spacy.load("en_core_web_sm")

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
# pip install pandas scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os
from sklearn.metrics import f1_score, confusion_matrix, classification_report

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\happy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\happy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\happy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


- Read Data
- Reduce the training set to speead up development. 

In [ ]:
OUTPUT_PATH = 'dataset/predicted_validation_data_from_rev2.csv'

In [4]:
## Read training data
df = pd.read_csv(r"dataset\data.csv")
# Reduce the training set to speed up development. 
print(df.head())
print(df['label'].value_counts())
print(df.isnull().sum())

   label                                              title  \
0      1  As U.S. budget fight looms, Republicans flip t...   
1      1  U.S. military to accept transgender recruits o...   
2      1  Senior U.S. Republican senator: 'Let Mr. Muell...   
3      1  FBI Russia probe helped by Australian diplomat...   
4      1  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017   
label
1    19999
0    19943
Name: count, dtype: int64
la

In [5]:
## Read validation data
val_data = pd.read_csv(r"dataset\validation_data.csv")
val_data.head(10)

,label,title,text,subject,date
0,2,UK's May 'receiving regular updates' on London...,LONDON (Reuters) - British Prime Minister Ther...,worldnews,"September 15, 2017"
1,2,UK transport police leading investigation of L...,LONDON (Reuters) - British counter-terrorism p...,worldnews,"September 15, 2017"
2,2,Pacific nations crack down on North Korean shi...,WELLINGTON (Reuters) - South Pacific island na...,worldnews,"September 15, 2017"
3,2,Three suspected al Qaeda militants killed in Y...,"ADEN, Yemen (Reuters) - Three suspected al Qae...",worldnews,"September 15, 2017"
4,2,Chinese academics prod Beijing to consider Nor...,BEIJING (Reuters) - Chinese academics are publ...,worldnews,"September 15, 2017"
5,2,Flames raced along train at west London statio...,LONDON (Reuters) - Flames engulfed one carriag...,worldnews,"September 15, 2017"
6,2,London police advise people to avoid area near...,LONDON (Reuters) - British police on Friday ad...,worldnews,"September 15, 2017"
7,2,London ambulance service sends hazardous area ...,LONDON (Reuters) - London s ambulance service ...,worldnews,"September 15, 2017"
8,2,Witness says injured in stampede at London sta...,LONDON (Reuters) - A woman at London s Parsons...,worldnews,"September 15, 2017"
9,2,UK says world will stand together against Nort...,LONDON (Reuters) - Britain said on Friday the ...,worldnews,"September 15, 2017"


## Split

In [6]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label']
)



Preprocess Text with spaCy + NLTK

In [12]:
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm", disable=["parser", "tagger"])

def preprocess_text(text, remove_named_entities=True):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    doc = nlp(text)
    ents = set(ent.text.lower() for ent in doc.ents) if remove_named_entities else set()
    tokens = [
        token.lemma_ for token in doc
        if token.lemma_ not in stop_words
        and token.is_alpha
        and token.text.lower() not in ents
    ]
    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\happy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [13]:
# Combine and clean
train_df['content'] = train_df['title'] + " " + train_df['text']
test_df['content'] = test_df['title'] + " " + test_df['text']

train_df['clean_content'] = train_df['content'].apply(preprocess_text)
test_df['clean_content'] = test_df['content'].apply(preprocess_text)


c:\Users\happy\anaconda3_32\Lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


KeyboardInterrupt: 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X_train_text = tfidf.fit_transform(train_df['clean_content'])
X_test_text = tfidf.transform(test_df['clean_content'])


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_df['subject_encoded'] = le.fit_transform(train_df['subject'])
test_df['subject_encoded'] = le.transform(test_df['subject'])

for df_ in [train_df, test_df]:
    df_['year'] = df_['date'].dt.year
    df_['month'] = df_['date'].dt.month
    df_['weekday'] = df_['date'].dt.weekday

X_train_meta = train_df[['subject_encoded', 'year', 'month', 'weekday']].values
X_test_meta = test_df[['subject_encoded', 'year', 'month', 'weekday']].values


## Data Preprocessing

## Feature Engineering

In [ ]:
from scipy.sparse import hstack

X_train = hstack([X_train_text, X_train_meta])
X_test = hstack([X_test_text, X_test_meta])
y_train = train_df['label']
y_test = test_df['label']

# Full dataset (for CV later)
X_all = hstack([tfidf.transform(df['title'] + " " + df['text'].apply(preprocess_text)), 
                df[['subject_encoded', 'year', 'month', 'weekday']].values])
y_all = df['label']


Train Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [ ]:
from sklearn.metrics import (
    accuracy_score, f1_score,
    classification_report, confusion_matrix
)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
import numpy as np

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X_all, y_all, cv=skf, scoring='f1')

print("5-Fold F1 Scores:", cv_scores)
print("Mean F1 Score:", np.mean(cv_scores))


Evaluation

# F1 Score

# Confusion Matrix

XGBoost

In [ ]:
%pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# Train
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Predict
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate
print("🔸 XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("🔸 XGBoost F1 Score:", f1_score(y_test, y_pred_xgb))
print("🔸 Confusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))
print("🔸 Classification Report:\n", classification_report(y_test, y_pred_xgb))


XG Boost cross val

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores_xgb = cross_val_score(xgb_model, X_all, y_all, cv=skf, scoring='f1')
print("🔸 XGBoost 5-Fold F1 Scores:", cv_scores_xgb)
print("🔸 Mean F1:", cv_scores_xgb.mean())


In [ ]:
##BERT

In [ ]:
%pip install transformers datasets

Prepare data for BERT

In [ ]:
from datasets import Dataset
from transformers import BertTokenizer

# Create datasets
bert_train_df = train_df[['clean_content', 'label']].rename(columns={'clean_content': 'text'})
bert_test_df = test_df[['clean_content', 'label']].rename(columns={'clean_content': 'text'})

train_ds = Dataset.from_pandas(bert_train_df)
test_ds = Dataset.from_pandas(bert_test_df)

# Tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

train_ds = train_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

# Set format for PyTorch
train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


Fine Tune BERT

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
)
trainer.train()


Evaluate BERT

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report

predictions = trainer.predict(test_ds)
y_pred_bert = np.argmax(predictions.predictions, axis=1)
y_true_bert = predictions.label_ids

print("🔷 BERT Accuracy:", accuracy_score(y_true_bert, y_pred_bert))
print("🔷 BERT F1 Score:", f1_score(y_true_bert, y_pred_bert))
print("🔷 BERT Classification Report:\n", classification_report(y_true_bert, y_pred_bert))


##OLD CODES for VAL DATA

In [ ]:
val_titles = val_data['title'].fillna("")
val_titles

Transform titles using the same vectorizer

In [ ]:
val_tfidf = vectorizer.transform(val_titles)

Predict labels

Replace label column

In [ ]:
val_data['label'] = predicted_labels
val_data

# Save the modified validation file

In [ ]:
val_data.to_csv(OUTPUT_PATH, index=False)

print(f"Predicted labels saved to {OUTPUT_PATH}")

cross validation